# Comprehensive Maximum Clique Algorithm Benchmarking

This notebook automatically runs all 11 C++ maximum clique algorithms on all datasets and generates detailed performance analysis.

**Algorithms tested:**
1. **Greedy** - Fast heuristic approximation
2. **Randomized** - Local search with random restarts
3. **Simulated Annealing** - Metaheuristic optimization
4. **Bron-Kerbosch** - Basic exact algorithm (skipped if >1000 vertices OR density >0.5)
5. **Tomita** - BK with pivot optimization (exact)
6. **Degeneracy BK** - BK with degeneracy ordering (exact)
7. **Östergård** - Branch-and-bound with coloring (exact)
8. **BBMC** - Bitset-based branch-and-bound (exact)
9. **CPU Optimized** - Bitset-optimized (exact, ≤8192 vertices)
10. **MaxCliqueDyn** - Tomita with dynamic coloring (exact)

**Output columns:**
- Dataset name, vertices, edges, density
- Max degree, average degree, degeneracy
- Clique size, wall-clock time, memory usage per algorithm

## 1. Import Libraries and Setup

In [15]:
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import glob

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 8)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

## 2. Configuration

In [16]:
# Configuration
BENCHMARK_SOURCE = "benchmark_comprehensive.cpp"
BENCHMARK_EXECUTABLE = "./benchmark_comprehensive"
DATASETS_FOLDERS = ["datasets/real_world", "datasets/synthetic", "datasets/benchmark"]
RESULTS_FOLDER = "benchmark_results"

# Create results folder
os.makedirs(RESULTS_FOLDER, exist_ok=True)

print("✅ Configuration loaded")
print(f"   Benchmark source: {BENCHMARK_SOURCE}")
print(f"   Results folder: {RESULTS_FOLDER}")


✅ Configuration loaded
   Benchmark source: benchmark_comprehensive.cpp
   Results folder: benchmark_results


## 3. Compile Benchmark

In [17]:
print("🔨 Compiling benchmark...")

compile_cmd = [
    "g++", "-std=c++17", "-O3",
    BENCHMARK_SOURCE,
    "-o", "benchmark_all"
]

result = subprocess.run(compile_cmd, capture_output=True, text=True)

if result.returncode != 0:
    print("❌ Compilation failed:")
    print(result.stderr)
    raise Exception("Compilation failed")
else:
    print("✅ Compilation successful!")
    print(f"   Executable: {BENCHMARK_EXECUTABLE}")

🔨 Compiling benchmark...
✅ Compilation successful!
   Executable: ./benchmark_comprehensive
✅ Compilation successful!
   Executable: ./benchmark_comprehensive


## 4. Discover Datasets

In [18]:
# Find all datasets
all_datasets = []
for folder in DATASETS_FOLDERS:
    if os.path.exists(folder):
        datasets = glob.glob(os.path.join(folder, "*.txt"))
        all_datasets.extend([(folder, os.path.basename(d), d) for d in datasets])

# Sort by category and name
all_datasets.sort()

print(f"📊 Found {len(all_datasets)} datasets:\n")
current_category = None
for category, name, path in all_datasets:
    if category != current_category:
        current_category = category
        category_name = category.split('/')[-1].upper()
        print(f"\n{category_name}:")
    file_size = os.path.getsize(path) / 1024  # KB
    print(f"  • {name:40s} ({file_size:8.1f} KB)")

📊 Found 22 datasets:


BENCHMARK:
  • C125.9.txt                               (    56.8 KB)
  • C250.9.txt                               (   250.2 KB)
  • brock400_2.txt                           (   552.8 KB)
  • ca-GrQc.txt                              (   185.9 KB)
  • email-Eu-core.txt                        (   156.0 KB)
  • frb30-15-01.txt                          (   855.6 KB)
  • frb35-17-01.txt                          (  1545.0 KB)
  • gen200_p0.9_44.txt                       (   156.2 KB)
  • keller4.txt                              (    80.6 KB)
  • p_hat300-1.txt                           (   100.0 KB)
  • queen5_5.txt                             (     0.0 KB)

REAL_WORLD:
  • facebook_combined.txt                    (  1006.9 KB)
  • twitter_combined.txt                     ( 48234.1 KB)

SYNTHETIC:
  • random_180v.txt                          (    20.9 KB)
  • rmat_er_large.txt                        (    74.4 KB)
  • rmat_er_small.txt                        (    17.1 K

## 5. Run Benchmarks on All Datasets

In [ ]:
import re
import sys

print(f"\n{'='*100}")
print(f"🚀 Starting comprehensive benchmark suite")
print(f"   Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"={'='*100}\n")

all_results = []
failed_datasets = []

for idx, (category, name, path) in enumerate(all_datasets, 1):
    print(f"\n[{idx}/{len(all_datasets)}] Running: {name}")
    print(f"   Category: {category.split('/')[-1]}")
    print(f"   Path: {path}")
    
    try:
        # Run benchmark
        print(f"   🔄 Starting benchmark execution...")
        sys.stdout.flush()
        start_time = datetime.now()
        
        # Use run() to capture all output at once (avoid streaming duplicates)
        result = subprocess.run(
            [BENCHMARK_EXECUTABLE, path],
            capture_output=True,
            text=True,
            timeout=600  # 10 minute timeout
        )
        
        end_time = datetime.now()
        total_time = (end_time - start_time).total_seconds()
        
        if result.returncode != 0:
            print(f"   ❌ Failed with return code {result.returncode}")
            print(f"   Error: {result.stderr[:200]}")
            failed_datasets.append(name)
            continue
        
        # Parse output
        lines = result.stdout.strip().split('\n')
        
        # Extract graph statistics
        vertices = edges = density = None
        for line in lines:
            if 'Vertices:' in line:
                try:
                    vertices = int(line.split(':')[1].strip().split()[0])
                except:
                    pass
            elif 'Edges:' in line:
                try:
                    edges = int(line.split(':')[1].strip().split()[0])
                except:
                    pass
            elif 'Density:' in line:
                try:
                    density = float(line.split(':')[1].strip().split()[0])
                except:
                    pass
        
        if vertices is None or edges is None or density is None:
            print(f"   ⚠️ Could not parse graph statistics")
            failed_datasets.append(name)
            continue
        
        print(f"   📊 Graph: {vertices:,} vertices, {edges:,} edges, density={density:.4f}")
        print(f"   ⏱️  Total benchmark time: {total_time:.2f}s")
        print(f"   ")
        print(f"   Algorithm Results:")
        
        # Parse algorithm results from RESULTS SUMMARY table
        # Format: Algorithm (30 chars), Clique Size (12 chars), Time (15 chars), Memory (15 chars)
        dataset_results = []
        in_results = False
        algo_count = 0
        
        for i, line in enumerate(lines):
            # Start parsing after "RESULTS SUMMARY:" and the separator line
            if 'RESULTS SUMMARY:' in line:
                in_results = False  # Will become true after next separator
                continue
            
            # Found the header separator, next lines are data
            if in_results == False and line.startswith('---') and i > 0:
                # Check if previous line has "Algorithm" header
                if i > 0 and 'Algorithm' in lines[i-1]:
                    in_results = True
                    continue
            
            # Stop at the next separator
            if in_results and line.startswith('---'):
                break
            
            if in_results and line.strip():
                # Parse: Algorithm(30), Clique Size(12), Time(15), Memory(15)
                if len(line) >= 42:
                    algo_name = line[:30].strip()
                    clique_str = line[30:42].strip()
                    time_str = line[42:57].strip()
                    
                    # Skip FAILED/SKIPPED entries
                    if clique_str in ['FAILED', 'SKIPPED', 'N/A']:
                        print(f"      ⏭️  {algo_name:30s} → SKIPPED")
                        continue
                    
                    # Replace spaces with underscores in algorithm names
                    algo_name_clean = algo_name.replace(' ', '_')
                    
                    try:
                        clique_size = int(clique_str)
                        time_taken = float(time_str)
                        
                        dataset_results.append({
                            'Dataset': name,
                            'Category': category.split('/')[-1],
                            'Vertices': vertices,
                            'Edges': edges,
                            'Density': density,
                            'Algorithm': algo_name_clean,
                            'CliqueSize': clique_size,
                            'Time(s)': time_taken,
                            'Success': True
                        })
                        
                        algo_count += 1
                        print(f"      ✓  {algo_name:30s} → Clique size: {clique_size:3d}, Time: {time_taken:8.4f}s")
                        
                    except (ValueError, IndexError) as e:
                        # If parsing fails, skip this line
                        continue
        
        if dataset_results:
            df_dataset = pd.DataFrame(dataset_results)
            all_results.append(df_dataset)
            
            print(f"   ")
            print(f"   ✅ Success! {algo_count} algorithms completed")
            
            # Show best result
            best = df_dataset.loc[df_dataset['CliqueSize'].idxmax()]
            print(f"   🎯 Best clique: {best['CliqueSize']} ({best['Algorithm']})")
            print(f"   ⚡ Fastest time: {df_dataset['Time(s)'].min():.4f}s ({df_dataset.loc[df_dataset['Time(s)'].idxmin()]['Algorithm']})")
        else:
            print(f"   ⚠️ No results parsed")
            failed_datasets.append(name)
        
    except subprocess.TimeoutExpired:
        print(f"   ⏱️ Timeout (>10 minutes)")
        failed_datasets.append(name)
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        failed_datasets.append(name)

print(f"\n{'='*100}")
print(f"✅ Benchmark suite completed!")
print(f"   Successful: {len(all_results)}/{len(all_datasets)}")
if failed_datasets:
    print(f"   Failed: {len(failed_datasets)} datasets")
    for name in failed_datasets:
        print(f"      • {name}")
print(f"={'='*100}")



🚀 Starting comprehensive benchmark suite
   Time: 2025-11-20 07:08:45


[1/22] Running: C125.9.txt
   Category: benchmark
   Path: datasets/benchmark/C125.9.txt
   🔄 Starting benchmark execution...


   📊 Graph: 125 vertices, 6,963 edges, density=89.8452
   ⏱️  Total benchmark time: 127.85s
   
   Algorithm Results:
      ✓  Greedy                         → Clique size:  29, Time:   0.0000s
      ✓  Randomized                     → Clique size:  29, Time:   0.0002s
      ✓  Simulated Annealing            → Clique size:  34, Time:   0.3627s
      ⏭️  Bron-Kerbosch                  → SKIPPED
      ✓  Tomita                         → Clique size:  34, Time:  66.2457s
      ✓  Degeneracy BK                  → Clique size:  34, Time:  46.7297s
      ✓  Ostergard                      → Clique size:  34, Time:  10.0284s
      ✓  BBMC                           → Clique size:  34, Time:   1.8832s
      ⏭️  CPU Optimized                  → SKIPPED
      ✓  MaxCliqueDyn                   → Clique size:  34, Time:   2.5850s
   
   ✅ Success! 8 algorithms completed
   🎯 Best clique: 34 (Simulated_Annealing)
   ⚡ Fastest time: 0.0000s (Greedy)

[2/22] Running: C250.9.txt
   Category: benchmark
 

## 6. Combine and Save Results

In [ ]:
# Combine all results into single dataframe
if len(all_results) > 0:
    df_all = pd.concat(all_results, ignore_index=True)
    
    # Save comprehensive CSV
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_path = os.path.join(RESULTS_FOLDER, f"benchmark_all_{timestamp}.csv")
    df_all.to_csv(csv_path, index=False)
    
    print(f"💾 Saved comprehensive results to: {csv_path}")
    print(f"   Total rows: {len(df_all)}")
    print(f"   Columns: {', '.join(df_all.columns)}")
    
    # Display sample
    print(f"\n📋 Sample results (first 10 rows):\n")
    display(df_all.head(10))
else:
    print("❌ No results to save")

## 7. Analysis: Performance by Algorithm

In [ ]:
if len(all_results) == 0:
    print("No results to analyze")
else:
    # Filter successful runs only
    df_success = df_all[df_all['Success'] == True].copy()
    
    print(f"\n{'='*100}")
    print(f"📊 ALGORITHM PERFORMANCE SUMMARY")
    print(f"{'='*100}\n")
    
    # Group by algorithm
    agg_dict = {
        'CliqueSize': ['mean', 'min', 'max'],
        'Time(s)': ['mean', 'median', 'min', 'max'],
        'Dataset': 'count'
    }
    
    algo_stats = df_success.groupby('Algorithm').agg(agg_dict).round(4)
    
    algo_stats.columns = ['_'.join(col).strip() for col in algo_stats.columns.values]
    algo_stats = algo_stats.rename(columns={'Dataset_count': 'Runs'})
    
    # Sort by average time
    algo_stats = algo_stats.sort_values('Time(s)_mean')
    
    display(algo_stats)
    
    # Save algorithm summary
    algo_csv = os.path.join(RESULTS_FOLDER, f"algorithm_summary_{timestamp}.csv")
    algo_stats.to_csv(algo_csv)
    print(f"\n💾 Algorithm summary saved to: {algo_csv}")

## 8. Analysis: Performance by Dataset

In [ ]:
if len(all_results) > 0:
    print(f"\n{'='*100}")
    print(f"📊 DATASET ANALYSIS")
    print(f"{'='*100}\n")
    
    # For each dataset, show best algorithm
    for dataset_name in df_all['Dataset'].unique():
        df_dataset = df_all[df_all['Dataset'] == dataset_name]
        df_dataset_success = df_dataset[df_dataset['Success'] == True]
        
        if len(df_dataset_success) == 0:
            print(f"⚠️ {dataset_name}: No successful runs")
            continue
        
        # Graph statistics (same across all algorithms)
        stats = df_dataset.iloc[0]
        print(f"\n📁 {dataset_name}")
        print(f"   Category: {stats['Category']}")
        print(f"   Graph: {stats['Vertices']:,} vertices, {stats['Edges']:,} edges")
        print(f"   Density: {stats['Density']:.4f}")
        
        # Best clique size
        max_clique = df_dataset_success['CliqueSize'].max()
        best_algos = df_dataset_success[df_dataset_success['CliqueSize'] == max_clique]
        print(f"   🎯 Best clique size: {max_clique}")
        print(f"      Found by: {', '.join(best_algos['Algorithm'].values)}")
        
        # Fastest algorithm
        fastest = df_dataset_success.loc[df_dataset_success['Time(s)'].idxmin()]
        print(f"   ⚡ Fastest: {fastest['Algorithm']} ({fastest['Time(s)']:.4f}s)")

## 9. Visualizations: Runtime Comparison

In [ ]:
if len(all_results) > 0:
    # Runtime comparison by algorithm
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Box plot of runtimes
    df_success_plot = df_success[df_success['Time(s)'] > 0]  # Filter out zero times
    
    algorithms = df_success_plot['Algorithm'].unique()
    data_to_plot = [df_success_plot[df_success_plot['Algorithm'] == algo]['Time(s)'].values 
                    for algo in algorithms]
    
    bp = ax.boxplot(data_to_plot, labels=algorithms, patch_artist=True)
    
    # Color boxes
    colors = plt.cm.Set3(range(len(algorithms)))
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_ylabel('Runtime (seconds)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Algorithm', fontsize=12, fontweight='bold')
    ax.set_title('Algorithm Runtime Distribution (All Datasets)', fontsize=14, fontweight='bold')
    ax.set_yscale('log')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    
    fig_path = os.path.join(RESULTS_FOLDER, f"runtime_boxplot_{timestamp}.png")
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Saved: {fig_path}")

## 10. Visualizations: Clique Size Comparison

In [ ]:
if len(all_results) > 0:
    # Average clique size by algorithm
    fig, ax = plt.subplots(figsize=(14, 6))
    
    clique_avg = df_success.groupby('Algorithm')['CliqueSize'].mean().sort_values(ascending=False)
    
    colors = ['#2ecc71' if val == clique_avg.max() else '#3498db' for val in clique_avg.values]
    
    ax.bar(range(len(clique_avg)), clique_avg.values, color=colors)
    ax.set_xticks(range(len(clique_avg)))
    ax.set_xticklabels(clique_avg.index, rotation=45, ha='right')
    ax.set_ylabel('Average Clique Size', fontsize=12, fontweight='bold')
    ax.set_title('Average Maximum Clique Size by Algorithm', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    
    fig_path = os.path.join(RESULTS_FOLDER, f"clique_size_avg_{timestamp}.png")
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Saved: {fig_path}")

## 11. Visualizations: Memory Usage

In [ ]:
if len(all_results) > 0:
    print("Memory usage data not available in this benchmark format.")

## 12. Heatmap: Algorithm Performance Across Datasets

In [ ]:
if len(all_results) > 0 and len(df_success['Dataset'].unique()) > 1:
    # Create pivot table for heatmap
    pivot_time = df_success.pivot_table(
        index='Algorithm', 
        columns='Dataset', 
        values='Time(s)',
        aggfunc='mean'
    )
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(18, 10))
    sns.heatmap(pivot_time, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax, 
                cbar_kws={'label': 'Runtime (seconds)'})
    ax.set_title('Algorithm Runtime Heatmap Across Datasets', fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Dataset', fontsize=12, fontweight='bold')
    ax.set_ylabel('Algorithm', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    fig_path = os.path.join(RESULTS_FOLDER, f"heatmap_runtime_{timestamp}.png")
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Saved: {fig_path}")

## 13. Final Summary

In [ ]:
if len(all_results) > 0:
    print(f"\n{'='*100}")
    print(f"✅ COMPREHENSIVE BENCHMARK COMPLETE")
    print(f"{'='*100}\n")
    
    print(f"📊 Statistics:")
    print(f"   Total datasets tested: {len(df_all['Dataset'].unique())}")
    print(f"   Total algorithm runs: {len(df_all)}")
    print(f"   Successful runs: {len(df_success)} ({len(df_success)/len(df_all)*100:.1f}%)")
    print(f"   Failed runs: {len(df_all) - len(df_success)}")
    
    print(f"\n🏆 Overall Best Performers:")
    
    # Fastest overall
    fastest_overall = df_success.groupby('Algorithm')['Time(s)'].mean().idxmin()
    fastest_time = df_success.groupby('Algorithm')['Time(s)'].mean().min()
    print(f"   ⚡ Fastest (avg): {fastest_overall} ({fastest_time:.4f}s)")
    
    # Best clique finder
    best_clique = df_success.groupby('Algorithm')['CliqueSize'].mean().idxmax()
    best_clique_size = df_success.groupby('Algorithm')['CliqueSize'].mean().max()
    print(f"   🎯 Best cliques (avg): {best_clique} ({best_clique_size:.2f})")
    
    print(f"\n📁 Results saved to: {RESULTS_FOLDER}/")
    print(f"   • Comprehensive CSV: benchmark_all_{timestamp}.csv")
    print(f"   • Algorithm summary: algorithm_summary_{timestamp}.csv")
    print(f"   • Visualizations: *.png")
    
    print(f"\n{'='*100}")
else:
    print("\n❌ No results generated")